# Automatizando CA Agile Central com python e a bibiloteca prally

### Introdução

Este notebook foi desenvolvido para interagir com as API's Rest do CA Agile Central versão 2.0.

Para executá-lo você precisa configurar o endereço do seu servidor, usuário e senha nas variáveis de ambiente do docker.

Para adicionar novas bibliotecas inclua-as na seção `imports`. Caso ocorra um erro `ModuleNotFoundError` e provável que a biblioteca não esteja instalada.

Para instalá-la, adicone na seção `Install` o comando de instalação (Exemplo: `!pip install nome_da_biblioteca`).

Para tornar a instalacao da biblioteca permanente (remover da seção `Install` e compartilhar com outros usuários), modifique o `Dockerfile` e crie uma nova imagem docker (ver README.md).

#### Install

In [ ]:
# Instale libs aqui

#### Imports

In [ ]:
# Imports
import sys, os
from pyral import Rally, rallyWorkset
import requests
import pyral
import datetime

#### Testes

In [ ]:
print ('Verificando as versões:')

print ('- Python ', sys.version)
assert sys.version_info >= (3,6), "Esperado versão python 3+"

print ('- pyral %s.%s.%s' % pyral.__version__)
assert pyral.__version__ >= (1,4), "Esperado versão pyral 1.4+"

print ('>>> OK.')

print('\nMy files:')
!ls -la

### Conectando com o servidor do CA Agile Central

In [ ]:
rally = Rally(os.environ['RALLY_SERVER'], os.environ['RALLY_USER'], os.environ['RALLY_PASSWORD'])

print ('CONECTADO')

### Função: Habilitar/Desabiltar um usuário por id

In [ ]:
def atualizar_usuario_por_id(object_id, disabled='true'):
    # valore válidos para disabled: true ou false, padrão true

    user_data = {"ObjectID" : object_id, "Disabled" : disabled}

    try:
        user = rally.update('User', user_data)
    except Exception as details:
        raise ValueError('Um erro ocorreu', details)

    return "Usuário %s atualizado para Disabled=%s" % (user.UserName, user.Disabled)

In [ ]:
# Testando a função
now = datetime.datetime.now()

try:
    print (('[%s] %s') % (str(now), atualizar_usuario_por_id('121212121212', 'true')))
except ValueError as err:
    print(err.args)

### Função: Pesquisar a lista de usuários

In [ ]:
def pesquisar_usuarios(query_criteria):
    
    response = rally.get('User', fetch=True, query=query_criteria)
    if response.errors:
        raise ValueError('Um erro ocorreu', response.errors)

    return response

### Pesquisando os usuários que serão desabilitados

In [ ]:
# Configure os parametros e a query
# Execute [Run] para testá-la
# Nenhum usuário será modificado
USERNAME = 'contratoca@email.com'
LASTACTIVEDATE='2018-07-06'
DISABLED='false'
QUERY_CRITERIA ='(((Disabled = %s) AND (LastActiveDate < %s)) AND (UserName != %s))' % (DISABLED, LASTACTIVEDATE, USERNAME)

index=0
now = datetime.datetime.now()

# Contém a lista de usuários
try:
    response=pesquisar_usuarios(QUERY_CRITERIA)
except ValueError as err:
    print(err.args)

print ('%s;%s;%s;%s;%s;%s' % ('ObjectID', 'UserName', 'FirstName', 'LastName', 'LastLoginDate', 'Disabled'))
for user in response:
    index+=1
    print ('%i;%s;%s;%s;%s;%s;%s' % (index, user.oid, user.UserName, user.FirstName, user.LastName, user.LastLoginDate, user.Disabled))

print('\n[%s] Total de usuários: %s' % (str(now), index))

### <span style="color:red">[ATENÇÃO]</span> Desabilitando os usuários

In [ ]:
# Usando o resultado da pesquisa acima (variável response)
# Ao executar [Run]
# OS USUÁRIOS SERÃO ATUALIZADOS
ans = input('Confirma? [S/N]: ')
if ans.lower() == 's':
    print('Desabilitando...')

    index=0
    now = datetime.datetime.now()

    for user in response:
        index+=1
        # Atualiza um usuário e imprime o resultado
        try:
            print(atualizar_usuario_por_id(user.oid, 'true'))
        except ValueError as err:
            print(err.args)

    print('\n[%s] Total de usuários desabilitados: %s' % (str(now), index))
else:
    print('Cancelado...')

## Referências

* [RallyRestToolkitForPython](https://github.com/RallyTools/RallyRestToolkitForPython#getting-started)
* [Rally Web Services API Documentation v2.0](https://rally1.rallydev.com/slm/doc/webservice/)
* [CA Agile Central Web Services API Documentation v2.0](https://rally1.rallydev.com/slm/doc/webservice/?version=v2.0)
* [Python toolkit for the Rally REST API](http://pyral.readthedocs.io/en/latest/overview.html)